Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [94]:
import numpy as np
import pandas as pd
import datetime

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

## Feature Engineering

In [95]:
df['interest_level'] = df['interest_level'].map({'low': 1, 'medium':2, 'high':3})

In [96]:
df['total_rooms'] = df['bathrooms'] + df['bedrooms']

## Train / Test Split

In [97]:
test = df[(df['created'] >= '2016-06-01')]
train = df[(df['created'] < '2016-06-01')]

## Linear Regression

In [98]:
from sklearn.linear_model import LinearRegression

In [99]:
# ESTABLISH BASELINE
target = 'price'
y_train = train[target]
y_test = test[target]

In [100]:
print(f'Mean Baseline (0 features): {y_train.mean():.2f}')

Mean Baseline (0 features): 3575.60


In [106]:
model = LinearRegression()

features = ['total_rooms', 'interest_level']
X_train = train[features]
X_test = test[features]

In [107]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Get the model's coefficients and intercept.

In [108]:
beta0 = model.intercept_
beta1, beta2 = model.coef_
print('Model equation: ')
print(f'y = {beta0:.2f} + {beta1:.2f} x1 + {beta2:.2f} x2')

Model equation: 
y = 2160.21 + 810.81 x1 + -578.25 x2


Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.

In [112]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [119]:
def print_metrics(target, predictions):
    mse = mean_squared_error(target, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(target, predictions)
    r2 = r2_score(target, predictions)
    print(f'Mean Squared Error: {mse:,.2f}')
    print(f'Root Mean Squared Error: {rmse:,.2f}')
    print(f'Mean Absolute Error: {mae:,.2f}')
    print(f'R-squared: {r2:,.4f}')

In [120]:
print_metrics(y_train, train_pred)

Mean Squared Error: 1,668,391.32
Root Mean Squared Error: 1,291.66
Mean Absolute Error: 856.69
R-squared: 0.4627


In [121]:
print_metrics(y_test, test_pred)

Mean Squared Error: 1,650,671.18
Root Mean Squared Error: 1,284.78
Mean Absolute Error: 865.82
R-squared: 0.4689


What's the best test MAE you can get? Share your score and features used with your cohort on Slack!

In [123]:
# This model fits slightly better
model = LinearRegression()

features = ['bedrooms', 'bathrooms', 'interest_level']
X_train = train[features]
X_test = test[features]
model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print_metrics(y_test, test_pred)

Mean Squared Error: 1,386,841.77
Root Mean Squared Error: 1,177.64
Mean Absolute Error: 792.66
R-squared: 0.5538


In [167]:
# Engineering lenghth of 'description' column
df['description_list'] = df['description'].astype(str).str.split()
df['description_list'] = df['description_list'].apply(lambda x: len(x))

In [170]:
# This model fits only slightly better than the previous

test = df[(df['created'] >= '2016-06-01')]
train = df[(df['created'] < '2016-06-01')]

target = 'price'
y_train = train[target]
y_test = test[target]

model = LinearRegression()

features = ['bedrooms', 'bathrooms', 'description_list', 'interest_level']
X_train = train[features]
X_test = test[features]
model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print_metrics(y_test, test_pred)

Mean Squared Error: 1,375,821.86
Root Mean Squared Error: 1,172.95
Mean Absolute Error: 785.99
R-squared: 0.5573
